In [23]:
#define document
#DKKD
Contents={}
Contents.update({"_Header":["Giấy ủy quyền này"]})
Contents.update({"BÊN ỦY QUYỀN":["Ông","Sinh ngày","Dân tộc","Quốc tịch","Chứng minh thư nhân dân số","Nơi đăng ký hổ khẩu thường trú","Chức vụ"]})
Contents.update({"BÊN ĐƯỢC ỦY QUYỀN":["Ông","Sinh ngày","Dân tộc","Quốc tịch","Chứng minh thư nhân dân số","Nơi đăng ký hổ khẩu thường trú","Chức vụ","1. NỘI DUNG VÀ PHẠM VI ỦY QUYỀN","2. TRÁCH NHIỆM CÁC BÊN","3. THỜI HẠN ỦY QUYỀN"]})
Contents.update({"_End":["BÊN ĐƯỢC ỦY QUYỀN"]})

In [24]:
Keywords0=list(Contents.keys())
for x in Keywords0:
    Contents[x].append("_end")
Contents

{'_Header': ['Giấy ủy quyền này', '_end'],
 'BÊN ỦY QUYỀN': ['Ông',
  'Sinh ngày',
  'Dân tộc',
  'Quốc tịch',
  'Chứng minh thư nhân dân số',
  'Nơi đăng ký hổ khẩu thường trú',
  'Chức vụ',
  '_end'],
 'BÊN ĐƯỢC ỦY QUYỀN': ['Ông',
  'Sinh ngày',
  'Dân tộc',
  'Quốc tịch',
  'Chứng minh thư nhân dân số',
  'Nơi đăng ký hổ khẩu thường trú',
  'Chức vụ',
  '1. NỘI DUNG VÀ PHẠM VI ỦY QUYỀN',
  '2. TRÁCH NHIỆM CÁC BÊN',
  '3. THỜI HẠN ỦY QUYỀN',
  '_end'],
 '_End': ['BÊN ĐƯỢC ỦY QUYỀN', '_end']}

In [25]:
# clean STR
import re
import numpy as np
def clean_str(s):
    s=s.strip()
    s=re.sub('[*&/`~@#$%\^&*]+', '', s)
    return s.lower()

def compare_str(s1,s2):
    n1=len(s1)
    n2=len(s2)
    f=np.zeros((n1+1,n2+1))
    for i in range(n1+1):
        f[i][0]=1
    for j in range(n2+1):
        f[0][j]=1    
    f[0][0]=0
    if(s1[0]!=s2[0]):
        f[1][1]=1
    for i in range(0,n1):
        for j in range(0,n2):
            if(s1[i]==s2[j]):
                f[i+1][j+1]=f[i][j]
            else:
                f[i+1][j+1]=min(f[i][j],f[i+1][j],f[i][j+1])+1
    
    return f[n1][n2]

def check_keyword(keyword,text,percent=0.3):
    keyword=clean_str(keyword)
    text=clean_str(text)
    n_keyword=len(keyword)
    n_text=len(text)
    similar=compare_str(keyword,text[:min(len(text),n_keyword)])
    return similar<=int(percent*min(n_keyword,n_text)),similar
    

def check_line(ct1,ct2,thresh_dis=20): #center 
    dis=abs(ct1[1]-ct2[1])
    return dis<thresh_dis

In [26]:
# list_text after remove noise:
import pickle
import cv2

img = cv2.imread("data/image.jpg")
print(img.shape)
(dt_boxes,list_texts)=pickle.load(open("res.pickle","rb+"))
n=len(dt_boxes)

(1149, 875, 3)


In [48]:
#get center and draw
heights=[]
def get_center(img,dt_boxes,draw=True):
    image=img.copy()
    color = (0, 255, 0)
    #y,x
    list_centers=[]
    for boxes in dt_boxes:
        
        heights.append(boxes[3][1]-boxes[0][1])
        heights.append(boxes[2][1]-boxes[1][1])
        x_center=int(sum(boxes[:,0])/4)
        y_center=int(sum(boxes[:,1])/4)
        list_centers.append((x_center,y_center))
        if(draw):
            # chieu kim dong ho
            image = cv2.line(image, (int(boxes[0][0]),int(boxes[0][1])), (int(boxes[1][0]),int(boxes[1][1])), color, 1) 
            image = cv2.line(image, (int(boxes[1][0]),int(boxes[1][1])), (int(boxes[2][0]),int(boxes[2][1])), color, 1) 
            image = cv2.line(image, (int(boxes[2][0]),int(boxes[2][1])), (int(boxes[3][0]),int(boxes[3][1])), color, 1)
            image = cv2.line(image, (int(boxes[3][0]),int(boxes[3][1])), (int(boxes[0][0]),int(boxes[0][1])), color, 1)
            image = cv2.circle(image,(y_center,x_center),1,color,1)
    if(draw):
        cv2.imwrite("output/image_with_boxes.jpg",image)
    return list_centers
list_centers=get_center(img,dt_boxes)

In [52]:
height_mean=sum(heights)/len(heights)
height_mean/=2

In [55]:
height_mean=int(height_mean)

In [7]:
list_texts

['CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM',
 'Độc lập ? Tự do ? Hạnh phúc',
 'GIẤY ỦY QUYỀN',
 'Căn cứ bộ luật dân sự;',
 'Căn cứ Luật doanh nghiệp 2014;',
 'Căn cử điều lệ của Công ty TNHH Thanh Huyền',
 'Giấy ủy quyền này được lập ngày/28/12/2020 tại trụ sở Công ty TNHH Thanh Huyền giữa các',
 'bên sau đây;',
 'BÊN ỦY QUYỀN',
 'Ông: Nguyễn Trường Giang',
 'Sinh ngày: 25/09/1985',
 'Dân tộc: Kinh',
 'Quốc tịch: Việt Nam',
 'Chứng minh thư nhân dân số 100860793 do Công an tỉnh Quảng Ninh cấp ngày 06/10/2010',
 'Nơi đăng ký hộ khẩu thường trú: Tổ 1, khu 1, phường Cao Xanh, thành phố Hạ Lòng, tỉnh',
 'Quảng Ninh',
 'Chức vụ: Chủ sở hữu kiêm Giám Đốc, là người đại diện theo Pháp luật của Công ty theo đăng ký',
 'kinh doanh thay đổi ngày 28/12/2020,',
 'BÊN ĐƯỢC ỦY QUYỀN:',
 'Ông: Nguyễn Xuân Ninh',
 'Sinh ngày: 12/05/1965',
 'Dân tộc: Kinh',
 'Quốc tịch: Việt Nam',
 'Chứng minh thư nhân dân số: 100638995 do Công an tỉnh Quảng Ninh cấp ngày 06/06/2008',
 'Noi đăng ký hộ khẩu thường trú: Tổ 2, k

In [56]:
# get_lines and sort
def get_lines(first,last,list_centers): # output list of list index which perline
    lines=[[first]]
    for i in range(first+1,last):

        if(check_line(list_centers[i],list_centers[i-1],height_mean)):
           lines[-1].append(i)
        else:
           lines.append([i])
    return lines

lines=get_lines(0,n,list_centers)
lines

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10, 11, 12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20, 21, 22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42, 43],
 [44],
 [45],
 [46]]

In [57]:
def sort_x(x):
  return list_centers[x][0]
def sort(center_ids):
    center_ids.sort(key=sort_x)
    return center_ids
ids=[]
for x in lines:
    for y in sort(x):
        ids.append(y)

In [58]:
list_texts_new=[]
list_boxes_new=[]
list_centers_new=[]
for i in ids:
    list_texts_new.append(list_texts[i])
    list_boxes_new.append(dt_boxes[i])
    list_centers_new.append(list_centers_new)

In [59]:

Keywords0

['_Header', 'BÊN ỦY QUYỀN', 'BÊN ĐƯỢC ỦY QUYỀN', '_End']

In [60]:
list_texts_new[10]
check_keyword(list_texts_new[10],Keywords0[1])

(False, 12.0)

In [61]:

begin = 0 

indexs={"_Header":0}


def get_index_keyword(keyword,list_text,begin,end):
    similar_min=1000
    index_final=-1
    for i in range(begin,end):
        check,similar=check_keyword(keyword,list_text[i])
        if check and similar < similar_min:
            similar_min=similar
            index_final=i
    return index_final

for keyword in Keywords0[1:-1]:
    index=get_index_keyword(keyword,list_texts_new,0,len(list_texts_new))
    if(index == -1 ):
        print(keyword,"  ",-1)
    else:
        print(keyword,"   :  ",index,list_texts_new[index])
    indexs[keyword]=index
    
if(Contents["_End"][0]!="None" and indexs[Keywords0[-2]]!=-1 ):
    index=get_index_keyword(Contents["_End"][0],list_texts_new,indexs[Keywords0[-2]]+1,len(list_texts_new))
    indexs["_End"]=index
else:
    indexs["_End"]=n

indexs
    

BÊN ỦY QUYỀN    :   8 BÊN ỦY QUYỀN
BÊN ĐƯỢC ỦY QUYỀN    :   18 BÊN ĐƯỢC ỦY QUYỀN:


{'_Header': 0, 'BÊN ỦY QUYỀN': 8, 'BÊN ĐƯỢC ỦY QUYỀN': 18, '_End': 42}

In [62]:
# get infor
indexs_final={}
for id,keyword0 in enumerate(Keywords0[:-1]):
    
    print("-------------------------",keyword0,"--------------------------------")
    begin=indexs[keyword0]
    if(begin!=-1):
        end=indexs[Keywords0[id+1]]
        indexs1={}
        keywords1=Contents[keyword0]
        for keyword in keywords1[:-1]:
            if(keyword[0]!="_"):
                index=get_index_keyword(keyword,list_texts_new,begin,end)
                if(index!=-1):
                    begin = index

    #                 print(keyword,"   ",list_texts_new[index])
                indexs1[keyword]=index
            else:
                indexs1[keyword]=begin
        indexs_final[keyword0]=indexs1
    
    print(indexs1)
     

------------------------- _Header --------------------------------
{'Giấy ủy quyền này': 6}
------------------------- BÊN ỦY QUYỀN --------------------------------
{'Ông': 9, 'Sinh ngày': 10, 'Dân tộc': 11, 'Quốc tịch': 12, 'Chứng minh thư nhân dân số': 13, 'Nơi đăng ký hổ khẩu thường trú': 14, 'Chức vụ': 16}
------------------------- BÊN ĐƯỢC ỦY QUYỀN --------------------------------
{'Ông': 19, 'Sinh ngày': 20, 'Dân tộc': 21, 'Quốc tịch': 22, 'Chứng minh thư nhân dân số': 23, 'Nơi đăng ký hổ khẩu thường trú': 24, 'Chức vụ': 26, '1. NỘI DUNG VÀ PHẠM VI ỦY QUYỀN': 27, '2. TRÁCH NHIỆM CÁC BÊN': 32, '3. THỜI HẠN ỦY QUYỀN': 37}


In [63]:
last=indexs["_End"]
for i in range(len(Keywords0)-2,-1,-1):
    indexs_final[Keywords0[i]]["_end"]=last
    if(indexs[Keywords0[i]]!=-1):
        last=indexs[Keywords0[i]]
    

In [64]:
indexs_final

{'_Header': {'Giấy ủy quyền này': 6, '_end': 8},
 'BÊN ỦY QUYỀN': {'Ông': 9,
  'Sinh ngày': 10,
  'Dân tộc': 11,
  'Quốc tịch': 12,
  'Chứng minh thư nhân dân số': 13,
  'Nơi đăng ký hổ khẩu thường trú': 14,
  'Chức vụ': 16,
  '_end': 18},
 'BÊN ĐƯỢC ỦY QUYỀN': {'Ông': 19,
  'Sinh ngày': 20,
  'Dân tộc': 21,
  'Quốc tịch': 22,
  'Chứng minh thư nhân dân số': 23,
  'Nơi đăng ký hổ khẩu thường trú': 24,
  'Chức vụ': 26,
  '1. NỘI DUNG VÀ PHẠM VI ỦY QUYỀN': 27,
  '2. TRÁCH NHIỆM CÁC BÊN': 32,
  '3. THỜI HẠN ỦY QUYỀN': 37,
  '_end': 42}}

In [65]:

for title in list(indexs_final.keys()):
    print("________________"+title+"________________")
    print()
    if(indexs[title]!=-1):
        index_title=indexs_final[title]
        keywords_title=list(index_title)
    
        for i in range(len(keywords_title)-1):
            if(index_title[keywords_title[i]]!=-1):
                begin= index_title[keywords_title[i]]
                end=index_title[keywords_title[i+1]]
                print("=============",keywords_title[i],"==============")
                for k in range(begin,end):
                    print(list_texts_new[k])
    print()
    print()
            
    
    

_________________Header________________

============= Giấy ủy quyền này ==============
Giấy ủy quyền này được lập ngày/28/12/2020 tại trụ sở Công ty TNHH Thanh Huyền giữa các
bên sau đây;


________________BÊN ỦY QUYỀN________________

============= Ông ==============
Ông: Nguyễn Trường Giang
============= Sinh ngày ==============
Sinh ngày: 25/09/1985
============= Dân tộc ==============
Dân tộc: Kinh
============= Quốc tịch ==============
Quốc tịch: Việt Nam
============= Chứng minh thư nhân dân số ==============
Chứng minh thư nhân dân số 100860793 do Công an tỉnh Quảng Ninh cấp ngày 06/10/2010
============= Nơi đăng ký hổ khẩu thường trú ==============
Nơi đăng ký hộ khẩu thường trú: Tổ 1, khu 1, phường Cao Xanh, thành phố Hạ Lòng, tỉnh
Quảng Ninh
============= Chức vụ ==============
Chức vụ: Chủ sở hữu kiêm Giám Đốc, là người đại diện theo Pháp luật của Công ty theo đăng ký
kinh doanh thay đổi ngày 28/12/2020,


________________BÊN ĐƯỢC ỦY QUYỀN________________

============= Ông